In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab05.5.ipynb")

# Lab 5.5: GLMs

#### The code and responses you need to write are represented by `...`. There is additional documentation for each part as you go along.

##### Please read carefully the introduction and the instructions for each problem.


## Collaboration Policy
You can submit the lab in pairs (groups of two, no more than two). **If you choose to work in a pair, please make sure to add your group member on Gradescope for both Lab 05.5 written submission and the Lab 05.5 code submission.**

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually and do not share your code with anyone other than your partner**. If you do discuss the assignments with people other than your partner please **include their names** in the cell below.

**Collaborators:**

## Submission

**For full credit, this assignment should be completed and submitted before Friday, Oct 11th, 2024 at 05:00 PM PST.**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from ipywidgets import interact, interactive
import itertools
import hashlib
from scipy.stats import poisson, norm, gamma
#!pip install pymc3
import statsmodels.api as sm
  
sns.set(style="dark")
plt.style.use("ggplot")

from pymc import *
import pymc as pm
import bambi as bmb
import arviz as az

## Review: GLMs [Adapts from Lab 05]

**If you completed Q3 in Lab 5, you can copy and paste your answers into the Lab 5.5 notebook (Q0) to avoid duplicating effort.**

Now, we will pivot to discussing frequentist and Bayesian approaches to generalized linear models. This question will be easier after Tuesday's lecture!

## Question 0: Atlantic Hurricane Season

With 30 named storms, the 2020 Atlantic hurricane season was the most active on record. Climate scientists argue that the culprit is human induced global warming. There is a an evergrowing body of research linking increased average temperatures and rising sea levels to more frequent, more intense and more destructive storms. 

In this lab we will investigate the number of named storms recorded since 1880, and we will argue that there is a statistically significant relationship between rising Sea Surface Temperature (SST) and the frequency of named storms.

For this lab we extracted the number of tropical storms from the [HURDAT Database](https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2019-052520.txt). We also extracted data on Sea Surface Temperatures from the [National Center for Atmosferic Research](https://climatedataguide.ucar.edu/climate-data/global-surface-temperature-data-gistemp-nasa-goddard-institute-space-studies-giss). 

### Load the data

In [ ]:
# No need to modify: Just run the code to load the data
data_source = "hurricane_data.csv"
df = pd.read_csv(data_source)
df = df[["Year", "Num_Storms", "Temp_Anomaly"]]
df.tail()

### Model Specifications

The `Num_Storms` column contains the number of named storms recorded each year between 1880 and 2019. The `Temp_Anomaly` column contains the deviation in yearly SST from the mean of 1951-1980.

In this question, to show that there is a statistically significant relationship between rising Sea Surface Temperature (SST) and the frequency of named storms, we will model the number of named storms in Year $i$ using **Poisson Regression**:

$$\lambda_i = e^{q_0 + q_1 X_i}$$ 

$$C_i \sim \text{Poisson}(\lambda_i),$$

where $X_i$ is the SST deviation in Year $i$, and $C_i$ is the number of named storms in year $i$.

This isn't something that we can easily solve from scratch, so we have to use software packages. In this question, we'll explore the two approaches to GLMs that we covered in class: 

1. **Q0(b): Frequentist Regression** using [`statsmodels.api`](https://www.statsmodels.org/stable/glm.html) 
2. **Q0(c) Bayesian Regression** via sampling using [`PyMC`](https://www.pymc.io/welcome.html) and [`Bambi`](https://bambinos.github.io/bambi/)

<!-- BEGIN QUESTION -->

## 0(a) Understanding Check

The model we described above is a GLM. What is "Linear" about this GLM model? What's the inverse link function? 


_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 0(b) Frequentist Regression

Let's start by considering the problem from a frequentist lens. To do this, we'll use the `statsmodels.api`, which allows us to create a model in just a few lines of code.

After fitting our model, we can call the `.summary()` method, and get a breakdown of our model and some details on how well it fit our data.

In [ ]:
# Fit Poisson GLM model where Temp_Anomaly is a covariate (exogenous variable): No need to modify
freq_model = sm.GLM(df["Num_Storms"], exog = sm.add_constant(df["Temp_Anomaly"]), 
                  family=sm.families.Poisson())
freq_res = freq_model.fit()
print(freq_res.summary())

<!-- BEGIN QUESTION -->

### 0(b)(i) Understanding the table

What variable does `Temp_Anomaly`'s `coef` in the table correspond to in our model? 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 0(b)(ii) Inspecting the results of fitting `freq_model`. 

Does the model suggest that increased SST relate to more storms? Is the influnce of SST on number of storms statistically significant?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 0(c) Bayesian Regression via PyMC

Now that we've done Poisson regression the frequentist way with the `statsmodels` package, let's try implementing it the Bayesian way! In this lab we'll explore two ways of doing this:
1. Building the model from scratch in `PyMC`
2. Using `Bambi`, a wrapper on `PyMC` that simplifies model construction

To start, let's build our model from scratch in `PyMC`. Unlike the `statsmodels` package, `PyMC` requires us to specify our model piece by piece. That means that, as with any Bayesian parameter estimation task, we have to distill our problem setup into a likelihood and prior before we can proceed.

### 0(c)(i) Unpacking $C_i \sim \text{Poisson}(\lambda_i)$

Recall the problem setup:

Our model involves the relationship between rising Sea Surface Temperature (SST) and the frequency of named storms, where:

$$\lambda_i = e^{q_0 + q_1 X_i}$$ 

$$C_i \sim \text{Poisson}(\lambda_i),$$

where $X_i$ is the SST deviation in Year $i$, and $C_i$ is the number of named storms in year $i$.

**In the cell below, Choose the option that best fills in the blank in the following statement:**

$C_i \sim \text{Poisson}(\lambda_i)$ represents our _____:

**A.** Prior

**B.** Likelihood

**C.** Posterior

Your answer should be a string, either `"A"`, `"B"`, or `"C"`.

In [ ]:
q0c_i = ...

In [ ]:
grader.check("q0c_i")

<!-- BEGIN QUESTION -->

### 0(c)(ii) Picking our prior(s)

Now, let's examine the parameter $\lambda_i = e^{q_0 + q_1 X_i}$. As discussed in lecture, $\lambda_i$ is comprised of a linear function of our observations ($q^TX$) and an inverse-link function ($e$). Given the setup of our problem, answer the following questions using 1 sentence each:
1. How many prior distributions do we need to define?
2. What parameters will we define these prior distributions for?
3. Since we don't have any prior information about these random variables, what distribution (i.e Normal, Beta, etc.) should we pick for their priors?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### 0(c)(iii) Defining our PyMC Model
Given your answers to `q2c_i` and `q2c_ii`, you're now ready to make your PyMC model! Fill out the code cell below to build your model and sample from the posterior.

**Note:** To pass the test, 
1. Do not remove the variables currently present in the model and/or add your own additional variables. Just fill in any ellipsis; the variables defined here are more than enough for you to solve the question!
2. Make sure the name parameter you pass to each `Distribution` object matches the variable name it's assigned to. Your answers should follow the following format: 

```
with pm.Model() as model:

    q0 = pm.Some_Distribution('q0', ...)
    q1 = pm.Some_Distribution('q1', ...)`
    ...
```

**Hint 1**: Remember that random variables can be added, subtracted, multiplied, etc. just like normal numbers!

**Hint 2**: Not all variables defined in a model context necessarily have to be defined using `pm.Some_Distribution(...)`. In particular, we do not need `pm.Deterministic` when defining `lam`, since we're not interested in its posterior samples. 

In [ ]:
with pm.Model() as model:
    q0 = ...
    q1 = ...

    lam = ...
    
    Y_obs = ...

    # DO NOT CHANGE THE SAMPLING ROUTINE
    trace = pm.sample(2000, chains = 4, random_seed = 0, return_inferencedata = True)

In [ ]:
grader.check("q0c_iii")

Now that we've ran PyMC, we can visualize the posterior distributions of $q_0$ and $q_1$ and find our estimates $\hat{q}_0$ and $\hat{q}_1$.

In [ ]:
az.plot_posterior(trace, round_to=3)
plt.show()

## 0(d) Bayesian Regression via Bambi

Whew! `q0(c)` took *a lot* of code just to build a simple Poisson regression. In practice, building models from scratch like this is usually unnecessary unless we're looking for a custom solution. Instead, we can rely on packages like `Bambi` that *wrap* the functionality of `PyMC` with a simpler interface purely designed for GLMs.

Using [`Bambi` documentation](https://bambinos.github.io/bambi/) as a guide, fill out the code cell below to fit a Bayesian Poisson Regression model on our data. 

**Note 1**: To pass the autograder, make sure you pass your model the `random_seed = 0` when you fit it!

**Note 2**: Notice that the `Bambi` package doesn't need us to specify a prior: when we don't specify a prior for our model parameters, Bambi automatically supplies a weakly informative one based on your data by default. For this question, we are okay with this behavior: **to pass the test, do not specify a prior on your parameters!**

In [ ]:
my_model = ...
my_model_samples = ...

az.plot_posterior(my_model_samples, round_to=3)
plt.show()

In [ ]:
grader.check("q0d")

<!-- BEGIN QUESTION -->

### 0(e) Understanding the plots
What the are x-axis and y-axis in each of the plots in 3d)? Your answer should be in terms of the parameters of our model.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 0(f) Comparison

Compare the results of `freq_model` in 0(b) with the plot in 0(d). Are the estimates of Frequentist and Bayesian Regression close to each other? 

_Type your answer here, replacing this text._

<!-- END QUESTION -->



In [ ]:
import matplotlib.image as mpimg
from otter.export import export_notebook
from os import path
from IPython.display import display, HTML
export_notebook("lab05.5.ipynb", filtering=True, pagebreaks=True)
if(path.exists('lab05.5.pdf')):
    img = mpimg.imread('chinchilla.jpg')
    imgplot = plt.imshow(img)
    imgplot.axes.get_xaxis().set_visible(False)
    imgplot.axes.get_yaxis().set_visible(False)
    plt.show()
    display(HTML("Download your PDF <a href='lab05.5.pdf' download>here</a>."))

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)